In [1]:
%matplotlib inline
import sys
import re
import nltk
import numpy as np
import pandas as pd
import pymorphy2
import sklearn
import itertools
from collections import Counter
from gensim.models.keyedvectors import KeyedVectors
from nltk.corpus import stopwords
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

C:\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
data = pd.read_csv('data/250ktendersList_norm.csv', sep='\t')
data.head()

,tender_id,tender_name,lots,otrasl,podortasl,okpd2,okpd2_name,norm_name,norm_lots,norm_okpd
0,50000048,Филиал Федерального бюджетного учреждения здра...,Услуги по оценке собственности,"Бизнес, финансы, страхование, маркетинг и реклама","Бухгалтерский учет, оценка, аудит",683116120,Услуги посреднические при оценке нежилого недв...,филиал_NOUN федеральный_ADJ бюджетный_ADJ учре...,услуга_NOUN оценка_NOUN собственность_NOUN,услуга_NOUN посреднический_ADJ оценка_NOUN неж...
1,50000049,МУНИЦИПАЛЬНОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ САРГАТСКОГ...,Поставка газа,Топливо и энергетика,Газ,352000000,"Газы горючие искусственные, услуги по распреде...",муниципальный_ADJ унитарный_ADJ предприятие_NO...,поставка_NOUN газа_NOUN,газа_NOUN горючий_ADJ искусственный_ADJ услуга...
2,50000050,АКЦИОНЕРНАЯ КОМПАНИЯ 'АЛРОСА' (ПУБЛИЧНОЕ АКЦИО...,НИГП. Разработка минералого-геохимических крит...,"Строительство, недвижимость и архитектура","Проектные работы, геодезия, картография, инжен...",711239000,"Услуги в области геологических, геофизических ...",акционерный_ADJ компания_NOUN алрос_NOUN публи...,разработка_NOUN геохимический_ADJ критерий_NOU...,услуга_NOUN область_NOUN геологический_ADJ гео...
3,50000149,АКЦИОНЕРНОЕ ОБЩЕСТВО 'ТРАНСНЕФТЬ - ЦЕНТРАЛЬНАЯ...,Техническое обслуживание теплоэнергетического ...,"Офис, дом",Услуги,353012000,Услуги по снабжению паром и горячей водой по т...,акционерный_ADJ общество_NOUN транснефть_NOUN ...,технический_ADJ обслуживание_NOUN теплоэнергет...,услуга_NOUN снабжение_NOUN паром_NOUN горячий_...
4,50000153,ПУБЛИЧНОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ЭНЕРГЕТИКИ И ЭЛ...,Открытый запрос предложений № 476-ОЗП/16-СМП н...,"Офис, дом",Мебель,310100000,Мебель для офисов и предприятий торговли,публичный_ADJ акционерный_ADJ общество_NOUN эн...,открытый_ADJ запрос_NOUN предложение_NOUN опре...,мебель_NOUN офис_NOUN предприятие_NOUN торговл...


In [4]:
# Формируем label для отрасли.
y1_encoder = LabelBinarizer()
y1_all = y1_encoder.fit_transform(data['otrasl'])
y1_names = y1_encoder.classes_

print(y1_all[0])
for i, name in enumerate(y1_names):
    print(i, name)

[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0 IT, компьютеры, связь
1 Безопасность
2 Бизнес, финансы, страхование, маркетинг и реклама
3 Бумажное производство, тара и упаковка
4 Деревообработка, лес
5 Издательство, полиграфия
6 Легкая промышленность
7 Машиностроение
8 Медицина, фармакология
9 Металлы, металлоизделия
10 Наука, исследования, образование
11 Офис, дом
12 Перевозки, логистика, таможня
13 Продовольствие, пищевая промышленность
14 Сельское хозяйство
15 Спорт, отдых, туризм
16 Строительство, недвижимость и архитектура
17 Сырье, полуфабрикаты
18 Топливо и энергетика
19 Транспорт
20 Химия
21 Экология
22 Электротехника


In [5]:
# Формируем label для подотрасли.

data['norm_podortasl'] = data['otrasl'] + ' - ' + data['podortasl']

y2_encoder = LabelBinarizer()
y2_all = y2_encoder.fit_transform(data['norm_podortasl'])
y2_names = y2_encoder.classes_

print(y2_all[0])
for i, name in enumerate(y2_names):
    print(i, name)

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0 IT, компьютеры, связь - Web, Интернет
1 IT, компьютеры, связь - Защита информации
2 IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
3 IT, компьютеры, связь - Программное обеспечение
4 IT, компьютеры, связь - Радиосвязь
5 IT, компьютеры, связь - Спутниковое телевидение
6 IT, компьютеры, связь - Телефония
7 IT, компьютеры, связь - Услуги провайдеров, хостинг, прочие услуги
8 Безопасность - Видеонаблюдение, Сигнализация, ОПС, СКД
9 Безопасность - Другое
10 Безопасность - Охрана и оборудование
11 Безопасность - Противопожарная безопасность
12 Бизнес, финансы, страхование, маркетинг и реклама - Банковское дело
13 Бизнес, финансы, страхование, маркетинг и реклама - Бухгалтерский учет, 

In [6]:
# Статистика встречаемости подотраслей
y2_stat = [(k, v) for (k, v) in Counter( data['norm_podortasl'] ).items()]
y2_stat.sort(key=lambda x: -x[1])
for k, v in y2_stat:
    print('%.2f %%: ' % (v * 100 / len(data)), k)

13.28 %:  Медицина, фармакология - Лекарственные препараты, витамины, расходные материалы
6.55 %:  Офис, дом - Услуги
5.10 %:  Строительство, недвижимость и архитектура - Ремонтные и строительные работы
5.09 %:  Медицина, фармакология - Медицинское оборудование, обслуживание, исследования
4.27 %:  Офис, дом - Канцелярские принадлежности
3.06 %:  IT, компьютеры, связь - Компьютеры, комплектующие, оргтехника, сети и сетевое оборудование
3.05 %:  Металлы, металлоизделия - Металлоизделия и металлоконструкции
2.74 %:  Наука, исследования, образование - Услуги
2.65 %:  Офис, дом - Бытовая техника
2.36 %:  Продовольствие, пищевая промышленность - Продукты питания
2.18 %:  Электротехника - Электронное оборудование, установки
2.05 %:  Строительство, недвижимость и архитектура - Строительные материалы, оборудование и инструменты
1.96 %:  Строительство, недвижимость и архитектура - Электроэнергия
1.88 %:  IT, компьютеры, связь - Программное обеспечение
1.77 %:  Электротехника - Электротехнические

In [7]:
# Обучать будем подотрасль
y_all = y2_all
y_names = y2_names
y_encoder = y2_encoder

for i in range(0, 3):
    y = np.argmax(y_all[i])
    print(y, y_names[y])

13 Бизнес, финансы, страхование, маркетинг и реклама - Бухгалтерский учет, оценка, аудит
102 Топливо и энергетика - Газ
91 Строительство, недвижимость и архитектура - Проектные работы, геодезия, картография, инженерные изыскания, кадастровые работы


In [19]:
words_total_list = list()

for i in range(len(data)):
    words_total_list.extend(str(data['norm_lots'][i]).split())
    words_total_list.extend(str(data['norm_okpd'][i]).split())

words_total_list = list(set(words_total_list))
words_total_list.sort()
words_total_dict = {w: i for (i, w) in enumerate(words_total_list)}
len(words_total_list)

28889

In [40]:
def build_texts_tensors(field_name, words_limit):
    tensors = np.zeros((len(data), words_limit), dtype=np.int)
    for i in range(len(data)):
        if i % 1000 == 0:
            sys.stdout.write('\rProcess: %d of %d' % (i, len(data)))
            sys.stdout.flush()
        words = set(str(data[field_name][i]).split())
        words = [w for w in words if w in w2v.vocab]
        words = words[:words_limit]
        for j, word in enumerate(words):
            tensors[i, j] = words_total_dict[word]
    print('. Complete.')
    print(tensors.shape)
    return tensors

lots_texts_all = build_texts_tensors('norm_lots', 8)
okpd_names_all = build_texts_tensors('norm_okpd', 8)

Process: 249000 of 249778. Complete.
(249778, 8)
Process: 249000 of 249778. Complete.
(249778, 8)


In [30]:
okpd_all = np.zeros((len(data), 410))

for i in range(len(data)):
    if i % 1000 == 0:
        sys.stdout.write('\rProcess: %d of %d' % (i, len(data)))
        sys.stdout.flush()
    okpd = data['okpd2'][i]
    okpd_all[i, okpd//10000000] = 1;
    okpd_all[i, 100 + okpd//100000%100] = 1;
    okpd_all[i, 200 + okpd//1000%100] = 1;
    okpd_all[i, 300 + okpd//10%100] = 1;
    okpd_all[i, 400 + okpd%10] = 1;

print('. Complete.')
okpd_all.shape

Process: 249000 of 249778. Complete.


(249778, 410)

In [41]:
# Маски для разделения выборок
np.random.seed(1)
rand_mask = np.random.rand(len(data))
split_ratio = 0.8
mask_tr = rand_mask <= split_ratio
mask_val = rand_mask > split_ratio

def split_by_masks(values_all):
    values_tr = values_all[mask_tr]
    values_val = values_all[mask_val]
    print(values_tr.shape, values_val.shape)
    return values_tr, values_val

okpd_tr, okpd_val = split_by_masks(okpd_all)
lots_texts_tr, lots_texts_val = split_by_masks(lots_texts_all)
okpd_names_tr, okpd_names_val = split_by_masks(okpd_names_all)
y_tr, y_val = split_by_masks(y_all)

(199924, 410) (49854, 410)
(199924, 8) (49854, 8)
(199924, 8) (49854, 8)
(199924, 131) (49854, 131)


In [42]:
import keras.layers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, TimeDistributed
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, Embedding
from keras.preprocessing.text import Tokenizer
from keras import regularizers

np.random.seed(1)

okpd_input_shape = (okpd_tr.shape[1],)
lot_text_input_shape = (lots_texts_tr.shape[1],)
okpd_name_input_shape = (okpd_names_tr.shape[1],)
out_len = y_tr.shape[1]
embedding_dict_size = len(words_total_list)
embedding_vect_length = 100

layer_okpd_input = Input(shape=okpd_input_shape)
layer_okpd = Dense(1024, activation='relu')(layer_okpd_input)
layer_okpd = Dropout(0.2)(layer_okpd)

layer_lot_text_input = Input(shape=lot_text_input_shape)
print(layer_lot_text_input)
layer_lot_text_embedding = Embedding(
    embedding_dict_size, 
    embedding_vect_length, 
    mask_zero=True, 
    trainable=True)
layer_lot_text = TimeDistributed(layer_lot_text_embedding)(layer_lot_text_input)
layer_lot_text = TimeDistributed(Dense(256, activation='relu'))(layer_lot_text)
layer_lot_text = TimeDistributed(Dropout(0.2))(layer_lot_text)
layer_lot_text = GlobalMaxPooling1D()(layer_lot_text)
layer_lot_text = Dense(256, activation='relu')(layer_lot_text)
layer_lot_text = Dropout(0.2)(layer_lot_text)

layer_okpd_name_input = Input(shape=okpd_name_input_shape)
layer_okpd_name_embedding = Embedding(
    embedding_dict_size, 
    embedding_vect_length, 
    mask_zero=True, 
    trainable=True)
layer_okpd_name = TimeDistributed(layer_okpd_name_embedding)(layer_okpd_name_input)
layer_okpd_name = TimeDistributed(Dense(512, activation='relu'))(layer_okpd_name)
layer_okpd_name = TimeDistributed(Dropout(0.2))(layer_okpd_name)
layer_okpd_name = GlobalMaxPooling1D()(layer_okpd_name)
layer_okpd_name = Dense(256, activation='relu')(layer_okpd_name)
layer_okpd_name = Dropout(0.2)(layer_okpd_name)

layer_merged = keras.layers.Concatenate()([layer_okpd, layer_lot_text, layer_okpd_name])
layer_merged = Dense(512, activation='relu')(layer_merged)
#layer_merged = Dense(256, activation='relu')(layer_okpd)
layer_merged = Dropout(0.2)(layer_merged)

layer_output = Dense(out_len, activation='softmax')(layer_merged)

model = Model(
    inputs=[layer_okpd_input, layer_lot_text_input, layer_okpd_name_input], 
    outputs=[layer_output])

model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

history = model.fit(
        [okpd_tr, lots_texts_tr, okpd_names_tr], 
        y_tr,
        batch_size=128,
        epochs=2)

model.evaluate([okpd_val, lots_texts_val, okpd_names_val], y_val, batch_size=128)

Tensor("input_17:0", shape=(?, 8), dtype=float32)


AssertionError: 

In [12]:
y_pred = model.predict([okpd_val, lots_texts_val, okpd_names_val], batch_size=128)
y_true = np.argmax(y_pred, axis=1) == np.argmax(y_val, axis=1)
print('accuracy:', np.mean(y_true))

accuracy: 0.81550126369


In [13]:
y_pred_all = model.predict([okpd_all, lots_texts_all, okpd_names_all], batch_size=128)


In [14]:
y_pred_all_argmax = np.argmax(y_pred_all, axis=1)

data_out = data.copy()

data_out['old'] = data_out['norm_podortasl']
data_out['new'] = [y_names[y_pred_all_argmax[i]] for i in range(len(data_out))]

y_false = np.argmax(y_pred_all, axis=1) != np.argmax(y_all, axis=1)
data_out = data_out.loc[y_false]

dropped_columns = [
    'norm_name', 
    'norm_lots', 
    'norm_okpd', 
    'norm_podortasl',
    'otrasl',
    'podortasl',
    'okpd2']

data_out = data_out.drop(dropped_columns, axis=1)
data_out.head()

,tender_id,tender_name,lots,okpd2_name,old,new
5,50000155,Филиал 'Астраханский судоремонтный завод ' АО ...,"Доработка системы автоматического управления, ...",Услуги по ремонту прочего оборудования,"Машиностроение - Ремонт, обслуживание, пуско-н...","Медицина, фармакология - Медицинское оборудова..."
6,50000234,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ 'ОРЕН...,Поставка клапанов обратных для нужд ООО 'Оренб...,Клапаны обратные,"Строительство, недвижимость и архитектура - Ре...","Машиностроение - Детали, запчасти, инструменты"
10,50000273,АКЦИОНЕРНОЕ ОБЩЕСТВО 'ДАЛЬНЕВОСТОЧНАЯ ГЕНЕРИРУ...,Установка железнодорожных весов ВТЭЦ-2,Услуги по ремонту прочего оборудования,"Строительство, недвижимость и архитектура - Ре...","Медицина, фармакология - Медицинское оборудова..."
17,50000499,ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕН...,Поставка продукции кабельной,Кабели высоковольтные малой мощности,Электротехника - Кабельно-проводниковая продукция,"Электротехника - Электронное оборудование, уст..."
28,50000588,ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО 'САХАЛИНАВТОДОРС...,закупка Концентрата минерального ГАЛИТ,"Соль и хлорид натрия чистый, вода морская","Продовольствие, пищевая промышленность - Специ...","Продовольствие, пищевая промышленность - Проду..."


In [15]:
#data_out.to_csv('data/250ktendersList_false.csv', sep='\t', index=False, encoding='utf-8')

with open('data/250ktendersList_false.txt', 'wt', encoding='utf-8-sig') as f:
    for i, row in data_out.iterrows():
        f.write('id     : %08d\n' % row['tender_id'])
        f.write('tender : %s\n' % row['tender_name'])
        f.write('lots   : %s\n' % row['lots'])
        f.write('okpd2  : %s\n' % row['okpd2_name'])
        f.write('results:\n')
        f.write('  old  : %s\n' % row['old'])
        f.write('  new  : %s\n' % row['new'])
        f.write('\n')
    